In [ ]:
!pip install librosa

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install keras-tuner

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   --------- ----------------------------- 30.7/129.1 kB 660.6 kB/s eta 0:00:01
   ------------------ -------------------- 61.4/129.1 kB 656.4 kB/s eta 0:00:01
   --------------------------------- ---- 112.6/129.1 kB 819.2 kB/s eta 0:00:01
   -------------------------------------- 129.1/129.1 kB 691.3 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import numpy as np
import librosa
from sklearn.svm import OneClassSVM
from keras.models import Model
from keras.layers import Dense, Input
import matplotlib.pyplot as plt


In [ ]:
base_dir = 'fan'
folders = ['id_00', 'id_02', 'id_04', 'id_06']
subfolders = ['abnormal', 'normal']


In [ ]:
def extract_features(file_name):
    y, sr = librosa.load(file_name, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)

    features = np.concatenate((np.mean(mfccs, axis=1),
                               np.mean(chroma, axis=1),
                               np.mean(spectral_contrast, axis=1)))
    return features

def load_data(base_dir, folders, subfolders):
    data = []
    labels = []
    for folder in folders:
        for subfolder in subfolders:
            path = os.path.join(base_dir, folder, subfolder)
            for file in os.listdir(path):
                file_path = os.path.join(path, file)
                features = extract_features(file_path)
                data.append(features)
                labels.append(0 if subfolder == 'normal' else 1)
    return np.array(data), np.array(labels)


In [ ]:
data, labels = load_data(base_dir, folders, subfolders)


C:\Users\User\AppData\Roaming\Python\Python311\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [ ]:
# Using only normal data for training
normal_data = data[labels == 0]

ocsvm = OneClassSVM(kernel='rbf', gamma='auto')
ocsvm.fit(normal_data)

# Predict anomalies
preds = ocsvm.predict(data)


In [ ]:
# Predict anomalies
preds = ocsvm.predict(data)

# Map One-Class SVM output to binary classes (0 for normal, 1 for abnormal)
mapped_preds = np.where(preds == -1, 1, 0)

# One-Class SVM evaluation
from sklearn.metrics import classification_report
print("One-Class SVM:")
print(classification_report(labels, mapped_preds, target_names=['normal', 'abnormal']))


One-Class SVM:
              precision    recall  f1-score   support

      normal       0.96      0.50      0.66      4075
    abnormal       0.40      0.94      0.56      1475

    accuracy                           0.62      5550
   macro avg       0.68      0.72      0.61      5550
weighted avg       0.81      0.62      0.63      5550



In [ ]:
# Auto-encoder evaluation
anomaly_preds = np.where(anomalies, 1, 0)
print("Auto-encoder:")
print(classification_report(labels, anomaly_preds, target_names=['normal', 'abnormal']))


Auto-encoder:
              precision    recall  f1-score   support

      normal       0.74      0.96      0.84      4075
    abnormal       0.41      0.08      0.13      1475

    accuracy                           0.72      5550
   macro avg       0.57      0.52      0.48      5550
weighted avg       0.65      0.72      0.65      5550



In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalize the data
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)

# Split the normalized data
normal_data_normalized = data_normalized[labels == 0]


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data_normalized, labels, test_size=0.2, random_state=42)

# Train One-Class SVM on training data
ocsvm.fit(X_train[y_train == 0])

# Predict on test data
test_preds = ocsvm.predict(X_test)
mapped_test_preds = np.where(test_preds == -1, 1, 0)

# Evaluate on test data
print("One-Class SVM Test Set Evaluation:")
print(classification_report(y_test, mapped_test_preds, target_names=['normal', 'abnormal']))

# Similarly, train and evaluate the auto-encoder model using the training and test sets


One-Class SVM Test Set Evaluation:
              precision    recall  f1-score   support

      normal       0.78      0.49      0.60       822
    abnormal       0.29      0.60      0.39       288

    accuracy                           0.52      1110
   macro avg       0.53      0.54      0.49      1110
weighted avg       0.65      0.52      0.54      1110



In [ ]:
from sklearn.ensemble import IsolationForest

# Assuming data_normalized contains your normalized feature data
iso_forest = IsolationForest(contamination=0.1, random_state=42)
iso_forest.fit(normal_data_normalized)

# Generate predictions
iso_preds = iso_forest.predict(data_normalized)

# Map predictions: 1 for normal, -1 for outlier/anomalous
mapped_iso_preds = np.where(iso_preds == 1, 0, 1)  # Assuming 0 is normal and 1 is abnormal
